Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# CSE 204 Lab 11: Unsupervised Learning - Clustering

<img src="https://raw.githubusercontent.com/adimajo/polytechnique-cse204-2019-releases/master/logo.jpg" style="float: left; width: 15%" />

[CSE204-2019](https://moodle.polytechnique.fr/course/view.php?id=7862) Lab session #11

J.B. Scoggins, Jesse Read, Adrien Ehrhardt, Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adimajo/polytechnique-cse204-2019-releases/blob/master/lab_session_11/lab_session_11.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/adimajo/polytechnique-cse204-2019-releases/master?filepath=lab_session_11%2Flab_session_11.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/adimajo/polytechnique-cse204-2019-releases/raw/master/lab_session_11/lab_session_11.ipynb)

## Introduction

In this lab, you will implement two unsupervised learning algorithms cluster data points based on similarity criteria: k-means, and spectral k-means.  While libraries such as scikit-learn provide facilities that implement these algorithms, they are simple enough for you to implement with numpy alone.  Before beginning, import the required packages.

In [ ]:
colab_requirements = [
    "matplotlib>=3.1.2",
    "numpy>=1.18.1",
    "nose>=1.3.7",
]
import sys, subprocess
def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())
        
if "google.colab" in sys.modules:
    for i in colab_requirements:
        run_subprocess_command("pip install " + i)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

### Datasets

Throughout this lab, you will use 3 simple datasets to test your algorithms.  Run the code below to visualize each dataset. As you can see, the first dataset consists of 4 gaussian-distributed clusters of points with equal variance.  The second represents two clusters, one stretched vertically, and one horizontally.  Finally, the last dataset represents 3 clusters distributed in rings.  For convenience, the three datasets are placed in a list called `datasets`.  In the rest of the lab, you will be asked to implement 2 clustering algorithms and run on them on these datasets.

In [ ]:
# Create a data set
N = 120

data1 = np.random.normal((0,0), (0.5,0.5) ,size=(N,2))
data1 = np.append(data1, np.random.normal((5,0), (0.5,0.5), size=(N,2)), axis=0)
data1 = np.append(data1, np.random.normal((0,5), (0.5,0.5), size=(N,2)), axis=0)
data1 = np.append(data1, np.random.normal((5,5), (0.5,0.5), size=(N,2)), axis=0)

data2 = np.random.normal((2,5), (0.25, 1), size=(N,2))
data2 = np.append(data2, np.random.normal((5,5), (1, 0.25), size=(N,2)), axis=0)

radii = np.random.normal(0,0.5,size=(N,1))
radii = np.append(radii, np.random.normal(4,0.5,size=(2*N,1)), axis=0)
radii = np.append(radii, np.random.normal(8,0.5,size=(3*N,1)), axis=0)
angles = np.random.uniform(size=(6*N,1))*2.0*np.pi
data3 = np.hstack([radii*np.cos(angles), radii*np.sin(angles)])

datasets = [data1, data2, data3]

fig, axes = plt.subplots(1,len(datasets), figsize=(10,3))
for i,data in enumerate(datasets):
    axes[i].scatter(data[:,0], data[:,1])

## Part 1: The k-Means Algorithm

k-means is one of the simplest unsupervised learning algorithms that solves the well known clustering problem. The algorithm defines an iterative process, where the following two steps take part at each iteration:
1. take each instance belonging to the dataset and assign it to the nearest centroid, and
2. re-calculate the centroids of each of the k clusters. 
Thus, the k centroids change their location step by step until no more changes are done.

More formally, suppose that we are given a dataset $X = \{x_1, x_2, \dots , x_n\}$, where each $x_i \in \mathbb{R}^d$. The goal of the k-means algorithm is to group the data into $k$ cohesive clusters, where $k$ is an input parameter of the algorithm. **Your task is to implement this algorithm**. Algorithm 1 gives the pseudocode.

___
### Algorithm 1: k-means

**Input**: The dataset $\mathbf{X} = \Big( \mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_n \Big)$ (where each $\mathbf{x}_i \in \mathbb{R}^d$) and the parameter $k$ (the number of clusters)<br>
**Output**: Clusters assignments (the cluster assigned to each element of $\mathbf{X}$)

1. Initialize cluster centroids ${\boldsymbol\mu}_1, \boldsymbol{\mu}_2, \ldots, \boldsymbol{\mu}_k$ by choosing $k$ instances of $\mathbf{X}$ randomly

**Repeat:**
2. Assign each instance $\mathbf{x}_i \in \mathbf{X}$ to the closest centroid, i.e., $c_i = \text{argmin}_j \|\mathbf{x}_i - \boldsymbol{\mu}_j\|$
3. Re-compute the centroids $\boldsymbol{\mu}_1, \boldsymbol{\mu}_2, \ldots, \boldsymbol{\mu}_k$ of each cluster based on $\boldsymbol{\mu}_j = \Big(\sum_{\mathbf{x} \in \mathbf{C}_j} \mathbf{x}\Big)/|\mathbf{C}_j|$, where $\mathbf{C}_j = \{\mathbf{x}_i  c_i = j\}$ the $j$-th cluster with $j=1, \ldots, k$ and $|\mathbf{C}_j|$ the size of the $j$-th cluster

**until** Centroids do not change (convergence)
___

In the algorithm above, $k$ is a parameter of the algorithm and corresponds to the number of clusters we want to find; the cluster centroids $\mu_j$ represent our current guesses for the positions of the centers of the clusters. To initialize the cluster centroids (in step 1 of the algorithm), we could choose $k$ training examples randomly, and set the cluster centroids to be equal to the values of these $k$ examples. Of course, other initialization methods are also possible, such as the [kmeans++ technique](https://en.wikipedia.org/wiki/K-means%2B%2B). To find the closest centroid, a distance (or similarity) function should be defined, and typically the Euclidean distance is used.

In [ ]:
def euclidean_distance(X1, X2):
    """
    Distance Function
    -----------------
    Computes the Euclidean distance between two arrays of points.
    
    Return
    ------
    A 2D n by m array where entry [i,k] is the distance 
    from the i-th point in X1 to the k-th point in X2.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def k_means(X, k):
    """
    k Means
    --------
    
    
    Parameters
    ----------
    
    X : an n-by-d matrix of inputs
    k : the number of clusters to find
    
    
    Algorithm:
    ----------
    
    0. Implement the euclidean distance function
    1. Initialize (choose) the centroids
    2. Implement a `while` loop such that, while centroids have not changed since the last iteration:
        - compute the distances of all points to each centroid
        - label each point (associate it with) the nearest centroid
        - recompute the centroids (i.e., average of points belonging to each centroid)
    
    Return
    ------
    
    z : a 1D vector of labels of length n (e.g. z[i] = c_j means "x_i is belongs to cluster c_j")
    iters : the number of iterations carried out until convergence
    """
    # Initialize (choose) the centroids
    # Iterate until convergence
    # YOUR CODE HERE
    raise NotImplementedError()
    return z, iters

To test your implementation, run the following code which will plot the 3 datasets, trying differente values of $k$. It will display the number of iterations until convergence (along with $k$ in the title).

In [ ]:
fig, axes = plt.subplots(3,len(datasets),figsize=(10,10))
for i,k in enumerate([2,3,4]):
    for j,data in enumerate(datasets):
        labels, iters = k_means(data, k)
        axes[i,j].scatter(data[:,0], data[:,1], c = labels, cmap='rainbow')
        axes[i,j].set_title('$k=%d$, iter$=%d$' % (k,iters))

Based on its notion of similarity, the problem of $k$-means clustering can be reduced to the problem of finding appropriate centroids. This, in turn, can be expressed as the task of minimizing the following objective function:
$$
     E(k) = \sum_{j=1}^{k} \sum_{\mathbf{x}_i \in \mathbf{C}_j}\| \mathbf{x}_i - \boldsymbol{\mu}_j \|^2.
$$

Thus, minimizing the above function is to determine suitable centroids $\mathbf{\mu}_j$ such that, if the data is partitioned into corresponding clusters $\mathbf{C}_j$, distances between data points and their closest cluster centroid become as small as possible.

The convergence of the $k$-means algorithm is highly dependent on the initialization of the centroids. It may converge to a local minimum of the objective function above. One way to overcome this problem is by executing the algorithm several times, with different initializations of the centroids. 

Another issue is how to determine the number of clusters ($k$) of the dataset. Intuitively, increasing $k$ without penalty, will always reduce the amount of error in the resulting clustering, to the extreme case of zero error if each data point is considered its own cluster (i.e., when $k=n$). One such method is known as the *elbow rule*. The idea is to examine  and compare the error given above for a number of cluster solutions.  In general, as the number of clusters increases, the *sum of squared errors* (SSE) should decrease because clusters are, by definition, smaller. A plot of the SSE against a series of sequential cluster levels (i.e., different values) can be helpful here. That is, an appropriate cluster solution could be defined as the one where the reduction in SSE slows dramatically. This produces an "elbow" in the plot of SSE against the different values of $k$. 

Bonus Task: Implement the elbow rule to find an appropriate value for $k$, as follows:

1. Run k-means clustering for values of $k=1,\ldots,10$. 
2. For each $k$, calculate the total intra-cluster error ($E(k)$, given above)
3. Plot the curve of $E(k)$ vs $k$.
4. Try to identify the location of a bend (elbow) in the plot -- this is generally considered as an indicator of the appropriate number of clusters.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part II: Spectral Clustering

Spectral clustering techniques make use of the *spectrum* (eigenvalues) of the similarity matrix of the data to perform dimensionality reduction before clustering in fewer dimensions. The similarity matrix is provided as an input and consists of a quantitative assessment of the relative similarity of each pair of points in the dataset.

Given a set of data points $\mathbf{x}_1, \ldots ,\mathbf{x}_n, \forall \mathbf{x}_i \in \mathbb{R}^d$ and some notion of similarity $s_{ij}$ between all pairs of data points $\mathbf{x}_i$ and $\mathbf{x}_j$, the intuitive goal of clustering is to divide the data points into several groups such that points in the same group are similar and points in different groups are dissimilar to each other. If we do not have more information than similarities between data points, a nice way of representing the data is in form of the similarity graph $G = (V,E)$. Each vertex $v_i$ in this graph represents a data point
$\mathbf{x}_i$. Two vertices are connected if the similarity $s_{ij}$ between the corresponding data points $\mathbf{x}_i$ and $\mathbf{x}_j$ is positive or larger than a certain threshold, and the edge is weighted by $s_{ij}$. The problem of clustering
can now be reformulated using the similarity graph: we want to find a partition of the graph such that the edges between different groups have very low weights (which means that points in different clusters are dissimilar from each other) and the edges within a group have high weights (which means that points within the same cluster are similar to each other).

### Creating the similarity graph

There are several popular constructions to transform a given set $\mathbf{x}_1, \ldots , \mathbf{x}_m, \forall \mathbf{x}_i \in \mathbb{R}^n$ of data points with pairwise
similarities $s_{ij}$ or pairwise distances $d_{ij}$ into a graph. When constructing similarity graphs the goal is to model the local neighborhood relationships between the data points. We will use the approach of the $k$-Nearest Neighbors graph. Here the goal is to connect vertex $\mathbf{x}_i$ with vertex $\mathbf{x}_j$ if $\mathbf{x}_j$ is among the $k$-nearest neighbors of $\mathbf{x}_i$. This definition leads to a directed graph, as the neighborhood relationship is not symmetric. The most common way to deal with this, is to simply ignore the directions of the edges; that is, we connect $\mathbf{x}_i$ and $\mathbf{x}_j$ with an undirected edge if $\mathbf{x}_i$ is among the $k$-nearest neighbors of $\mathbf{x}_j$ or if $\mathbf{x}_j$ is among the $k$-nearest neighbors of $\mathbf{x}_i$. The resulting graph is what is usually called the $k$-nearest neighbors graph.

### The algorithm

The pseudocode of the spectral clustering algorithm is given as follows. In spectral clustering, the data is projected into a lower-dimensional space (the spectral/eigenvector domain) where they are easily separable, say using $k$-means. 

Given dataset $\mathbf{X}=\{ \mathbf{x_1}, \mathbf{x_2}, \ldots, \mathbf{x_m}\}$, where  each $\mathbf{x}_i \in \mathbb{R}^n$ and parameter $k$:

1. Construct the similarity graph $G$ as described above. Let 
    * $\mathbf{W}$ be the adjacency matrix of this graph.
    * $\mathbf{D}$ be the diagonal degree matrix of graph $G$, ie $\mathbf{D}_{ii} = \sum_j \mathbf{W}_{ij}$ and $\mathbf{D}_{ij} = 0$ for $i \neq j$.
2. Compute the Laplacian matrix $\mathbf{L} = \mathbf{D} - \mathbf{W}$. 
3. Apply eigenvalue decomposition to the Laplacian matrix $\mathbf{L}$ 
4. Select the eigenvectors that correspond to $k$ smallest eigenvalues. Let $\mathbf{U}$ contain these corresponding eigenvectors in its columns.
5. Apply $k$-means to $\mathbf{U}$ (as if the rows were instances), thus finding clusters $\mathbf{C}_1, \mathbf{C}_2, \ldots, \mathbf{C}_k$.

**Your Task** Implement the algorithm. *Hint*: Use the distance function and the $k$-means implementation you wrote in Task 1.

*Python hints:* to find the `k` nearest neighbors of $x_i$, you can use the function `np.argsort` with the array that stores `d(x_i, x_j)`. You can use `eigenvalues, eigenvectors = np.linalg.eig(L)` to obtain the eigenvalues and eigenvectors of a (square) matrix `L`.


In [ ]:
def spectral_k_means(X, k, k_nn):
    '''
        Spectral Clustering
        -------------------
        
        Parameters
        ----------
        
        X : the data
        k : the number of clusters to find
        k_nn : the number of k nearest-neighbours to consider in the graph construction
        
        Returns
        -------
        
        The same numbers as your k-means method above
    '''
    # YOUR CODE HERE
    raise NotImplementedError()
    return k_means(U, k)
    

To test your implementation, run the following code which will plot the 3 datasets, trying different values of $k$. It will display the number of iterations (done by $k$-means) until convergence (along with $k$ in the title).

In [ ]:
fig, axes = plt.subplots(3, len(datasets), figsize=(10, 10))
for i, k in enumerate([2, 3, 4]):
    for j, data in enumerate(datasets):
        labels, iters = spectral_k_means(data, k, 10)
        axes[i,j].scatter(data[:,0], data[:,1], c=labels, cmap='rainbow')
        axes[i,j].set_title('$k=%d$, iter$=%d$' % (k, iters))

## Task 3: Gaussian Mixture Model (Bonus Task)

Suppose you observe and measure a number of beetles, recording their length in centimeters -- given below in the array `x`. You are curious to investigate if you have recorded more than one species. In fact you believe you are studying two distinct species. You decide to use Gaussian Mixture Models on the data to elaborate on this hypothesis and gain further insight into the beetle population. 

**Task: Implement Gaussian Mixture Models**

Note: Plotting code is provided. 
Hint: See the lecture slides regarding pseudocode and outline. Note that you may obtain slightly different results each time you run the algorithm.

In [ ]:
# beetle lengths
x = np.array([1.57, 1.16, 1.30, 0.26, 0.20, 0.43, 0.48,  0.34,  0.44,  0.61, 1.80, 1.40,  1.10,  1.25, 1.69], dtype=float)
# no. of beetles
N = len(x)
# no. species
K = 2

# responsibilities (beetle i pertains r[i,k]-much to species k)
r = np.random.random((N,K))
# normalized
r[:,1] = np.ones(N) - r[:,0]

## evaluate (Gaussian) pdf 'g' at point 'x' under mean m, sd s
def g(x, m = 0.0, s = 1.0):
    return np.exp(-(x - m)**2/(2*s))/np.sqrt(2*s*np.pi)

# Initialize parameters to random values
m = np.array([0.0, 1.0])
s = np.array([1.0] * 2)

# Initialize weights (prior probabilities) aka pi
w = np.ones(K)/N

# EM ALGORITHM
T = 100 # max iterations
for t in range(0, T):
    # YOUR CODE HERE
    raise NotImplementedError()
        
fig = plt.figure()
xx = np.linspace(0, 2, num=100)      # points to plot
plt.plot(xx,g(xx,m[0],s[0]),'r-',label="$g_1$")
plt.plot(xx,g(xx,m[1],s[1]),'b-',label="$g_2$")
plt.plot(xx,w[0]*g(xx, m[0], s[0]) + w[1] * g(xx, m[1], s[1]), 'm:', label="mixture")
c = np.sum(m)/2.
y = (x < c)*1 
plt.legend()
plt.scatter(x,np.zeros(N), c=y, label="beetles")
plt.show()
